In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import os

# Loading the merged dataset
ords_prods_merge = pd.read_pickle(r"/Users/boukaskasbrahim/Desktop/Achievement4_Project/Prepared Data/ords_prods_merge.pkl")


In [2]:
# Aggregation for entire dataframe
dept_order_mean = ords_prods_merge.groupby('department_id')['order_number'].mean()

print(dept_order_mean)


department_id
1     15.457687
2     17.277920
3     17.179756
4     17.811403
5     15.213779
6     16.439806
7     17.225773
8     15.340520
9     15.895474
10    20.197148
11    16.170828
12    15.887622
13    16.583304
14    16.757377
15    16.165037
16    17.663250
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64


### Mean Order Numbers by Department

The table above shows the **average number of orders** placed per department across the **entire dataset**.  

Compared to the earlier subset analysis, these results are more reliable since they include all available records.  
- The averages vary across departments, generally ranging between ~15 and ~23 orders.  
- Some departments (e.g., ID 21) have noticeably higher average order numbers than others.  
- This suggests that certain departments consistently attract more repeat orders, while others see fewer.  

By analyzing the full dataset, we reduce sampling bias and get a more accurate picture of customer behavior in each department.


In [4]:
# Creating a max_order column
ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform(np.max)

# Creating loyalty_flag based on criteria
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal Customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & 
                     (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular Customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New Customer'

# Checking results
ords_prods_merge['loyalty_flag'].value_counts()


/var/folders/fh/z04k5mcd3xz6ftdmhk_5j1m00000gn/T/ipykernel_22623/4196074309.py:2: FutureWarning: The provided callable <function max at 0x10817fa60> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby('user_id')['order_number'].transform(np.max)


loyalty_flag
Regular Customer    15891077
Loyal Customer      10293737
New Customer         6249398
Name: count, dtype: int64

### Loyalty Flag Results

The loyalty flag was successfully created using the maximum number of orders per user:

- **Loyal Customers**: more than 40 orders  
- **Regular Customers**: 11–40 orders  
- **New Customers**: 10 or fewer orders


The counts show that the majority of users are **Regular Customers**, followed by **Loyal Customers** and then **New Customers**.  
This segmentation provides valuable insights 


In [5]:
# Descriptive statistics for product prices by loyalty_flag
price_stats = ords_prods_merge.groupby('loyalty_flag')['prices'].describe()

print(price_stats)


                       count       mean         std  min  25%  50%   75%  \
loyalty_flag                                                               
Loyal Customer    10293737.0  10.388747  327.864108  1.0  4.2  7.4  11.2   
New Customer       6249398.0  13.294370  597.301692  1.0  4.2  7.4  11.3   
Regular Customer  15891077.0  12.496203  539.478009  1.0  4.2  7.4  11.3   

                      max  
loyalty_flag               
Loyal Customer    99999.0  
New Customer      99999.0  
Regular Customer  99999.0  


### Product Prices by Loyalty Category

The table shows descriptive statistics for product prices across the three loyalty groups:

- **Loyal Customers**: average price ≈ 10.39  
- **Regular Customers**: average price ≈ 12.50  
- **New Customers**: average price ≈ 13.29  

Across all groups, the median price (≈ 7.4) and the quartiles (≈ 4.2 and 11.2–11.3) are very similar.  

**Insights:**
- Loyal customers tend to purchase slightly cheaper products on average.  
- Regular and new customers lean toward somewhat higher-priced items.  
- Since the distributions are close, loyalty appears to be more about **order frequency** rather than choosing premium products.  

This means Instacart might want to encourage loyal customers with **bundle offers or discounts**, while targeting new customers with **higher-value product promotions**.

In [8]:
# Step 1: Calculating the mean price per user
ords_prods_merge['avg_price'] = ords_prods_merge.groupby('user_id')['prices'].transform('mean')

# Step 2: Creating spending_flag
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spending_flag'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_flag'] = 'High spender'

# Checking distribution
ords_prods_merge['spending_flag'].value_counts()


spending_flag
Low spender     31798751
High spender      635461
Name: count, dtype: int64

### Spending Flag Results

The spending flag was created based on each user’s **average product price**:  
- **Low spender**: avg < 10  
- **High spender**: avg ≥ 10  

The results show that the majority of users fall into the **Low spender** category (≈ 31.8 million), while a much smaller group are **High spenders** (≈ 635k).  

**Insights:**  
- Most Instacart customers prefer lower-priced items.  
- The high spender group is relatively small but could represent a valuable target for premium product promotions.


In [9]:
#  1: Median days_since_prior_order per user
ords_prods_merge['median_days'] = ords_prods_merge.groupby('user_id')['days_since_prior_order'].transform('median')

#  2: Assigning frequency_flag
ords_prods_merge.loc[ords_prods_merge['median_days'] > 20, 'frequency_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_days'] > 10) & 
                     (ords_prods_merge['median_days'] <= 20), 'frequency_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_days'] <= 10, 'frequency_flag'] = 'Frequent customer'

#  3: Checking distribution
ords_prods_merge['frequency_flag'].value_counts()


frequency_flag
Frequent customer        21577409
Regular customer          7217134
Non-frequent customer     3639669
Name: count, dtype: int64

In [10]:
# Exporting enriched dataframe as pickle
ords_prods_merge.to_pickle(r"/Users/boukaskasbrahim/Desktop/Achievement4_Project/Prepared Data/ords_prods_merge_enriched.pkl")
